In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
import time 


In [2]:
options = Options()
options.add_argument("--start-maximized")
service = Service()

driver = webdriver.Chrome(service=service, options=options)
driver.get("https://www.kayak.com/flights")
time.sleep(5)
print(driver.title)


Cheap Flights, Airline Tickets & Airfare Deals | KAYAK
